# LDA Topic Model 

This code is pulled from https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/. 

We apply an LDA Topic Model to reviews of companies on Glassdoor to understand what topics/categories people talk about when describing their work place / work experience.  

## Import Libraries

In [1]:
import pandas as pd

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/connorgilmore/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint


# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
#import pyLDAvis.gensim  # don't skip this
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

/Users/connorgilmore/opt/anaconda3/lib/python3.7/site-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable
/Users/connorgilmore/opt/anaconda3/lib/python3.7/site-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping
/Users/connorgilmore/opt/anaconda3/lib/python3.7/site-packages/scipy/io/matlab/mio5.py:98: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


In [4]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

## Import Data

In [5]:
reviews_df = pd.read_csv('reviews.csv')

In [6]:
reviews_df.head()

,Unnamed: 0,company,pros,cons
0,0,Uber,"Hours, pay, management and environment",I wouldn’t say I had any cons working at this job
1,1,Uber,Work your very own schedule,Sometimes the pay can be unbalanced
2,2,Uber,"Uber stock, good benefits, good experience","Bad commission structure, limited upside"
3,3,Uber,Decent pay for the work you do.,The customer service department doesnt take th...
4,4,Uber,Smart people\r\nLots of resources\r\nGreat com...,Fast-moving org - sometimes a little too fast ...


In [7]:
reviews_df['combined'] = reviews_df['pros'] + reviews_df['cons']

## Data Prep

In [8]:
data = reviews_df['pros']
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

In [9]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [10]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [11]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
#nlp = spacy.load('en', disable=['parser', 'ner'])
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

## LDA

In [12]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [13]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        #model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model = lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=num_topics)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=5, limit=10, step=1)

## Visualize Performance Metrics

In [ ]:
limit=10; start=5; step=1;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

Optimal Model

In [ ]:
optimal_model = model_list[1]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=15))

In [15]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=10)
model_topics = lda_model.show_topics(formatted=False)
pprint(lda_model.print_topics(num_words=30))

[(0,
  '0.111*"work" + 0.067*"life" + 0.065*"balance" + 0.044*"good" + '
  '0.025*"company" + 0.021*"great" + 0.019*"people" + 0.017*"culture" + '
  '0.015*"benefit" + 0.013*"environment" + 0.011*"opportunity" + '
  '0.011*"employee" + 0.010*"flexible" + 0.009*"excellent" + 0.009*"hour" + '
  '0.008*"really" + 0.008*"lot" + 0.008*"focus" + 0.007*"team" + '
  '0.007*"leadership" + 0.006*"care" + 0.006*"well" + 0.006*"make" + '
  '0.006*"pay" + 0.005*"time" + 0.005*"nice" + 0.005*"flexibility" + '
  '0.005*"schedule" + 0.005*"management" + 0.004*"amazing"'),
 (1,
  '0.104*"good" + 0.036*"work" + 0.025*"people" + 0.023*"great" + 0.022*"pay" '
  '+ 0.021*"benefit" + 0.020*"nice" + 0.017*"salary" + 0.014*"bonus" + '
  '0.013*"manager" + 0.013*"project" + 0.012*"culture" + 0.012*"learn" + '
  '0.011*"job" + 0.010*"company" + 0.008*"pretty" + 0.008*"amazing" + '
  '0.007*"change" + 0.007*"colleague" + 0.006*"willing" + 0.006*"always" + '
  '0.006*"development" + 0.006*"smart" + 0.006*"lot" + 

## Visualize Topics

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(optimal_model, corpus, id2word)
vis

## Get Topic Distribution

In [ ]:
ldaModel.get_document_topics(bagOfWordOfADocument, minimum_probability=0.0)